In [1]:
import time
import random
import math
import os
import sys
import numpy as np


In [2]:
def time_since(since):
    now = time.time()
    s = now - since
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)

def req(method, path, json):
    print(f'req({method}, {path}, {json})')
    return None

In [3]:
from torchnet.logger import VisdomPlotLogger, VisdomLogger, VisdomTextLogger
from tqdm import tqdm
import io
import json

In [15]:
# Start a job

class Job:
    def __init__(self, name, params={}, hostname='localhost', port=8890, n_epochs=10):
        self.name = name
        self.params = params
        self.hostname = hostname
        self.avg_train_loss_logger = VisdomPlotLogger(
        'line', port=port, opts={'title': f'{name}_Train Loss'})
        self.job_logger = VisdomTextLogger(port=port, opts={'title': f'{name}_Job Info'})
        self.log_logger = VisdomTextLogger(port=port, opts={'title': f'{name}_Train Loss Log'}, update_type='APPEND')
        self.pbar = tqdm(total=n_epochs, ncols=60, mininterval=1.0, ascii=True, file=io.StringIO())
        self.progress_logger = VisdomTextLogger(port=port, opts=dict(title=f'{name}_Progress Bar',
            width=500,
            height=50,
        ))

        self.start()

    def start(self):
#         j = {'name': self.name, 'params': self.params, 'hostname': self.hostname}
        j_str = json.dumps(self.params, indent=4)
        j_str = j_str.replace('\n', '<br/>\n').replace(' ', '&nbsp;')
        self.job_logger.log(f'{j_str}')

        self.job_id = '#job_id'# body['id']
        print("Starting job %s at %s" % (self.job_id, time.strftime('%Y-%m-%d %H:%M:%S', time.localtime())))
        self.start_time = time.time()

        self.log_every = 50
        self.plot_every = 50
        self.loss_avg = 0

    def stop(self, ):
        j = {'status': 'done'}
        print(j)

    def log(self, l):
        def time_since(since):
            now = time.time()
            s = now - since
            m = math.floor(s / 60)
            s -= m * 60
            return '%dm %ds' % (m, s)
        # print('[log] %s' % time_since(self.start_time), l)
        self.log_logger.log(l)

    def plot(self, x, y):
        # print(f'[plot] ({x}, {y})')
        self.avg_train_loss_logger.log(x, y)

    def record(self, e, loss):
        self.loss_avg += loss

        if e > 0 and e % self.log_every == 0:
            self.log('(%s) %.4f' % (e, loss))

        if e > 0 and e % self.plot_every == 0:
            self.plot(e, self.loss_avg / self.plot_every)
            self.loss_avg = 0

    def update_progress(self, ):
        self.pbar.update(1)
        text = f'{self.pbar}'.replace(' ', '<font color="white">#</font>')
        self.progress_logger.log(text)

In [16]:
n_epochs = 10
job = Job('seq2seq-translate', {
    'attn_model': 'ala',
    'n_layers': 3,
    'dropout': 0.3,
    'hidden_size': 1000,
    'learning_rate': 0.001,
    'clip': 0.1,
    'teacher_forcing_ratio': 0.1,
    'decoder_learning_ratio': 0.2,
}, n_epochs)
job.plot_every = 500
job.log_every = 100

Starting job #job_id at 2018-05-06 22:41:00


In [17]:
epoch = 1

while epoch < n_epochs:
    loss = random.random()
    job.record(epoch, loss)
    print(f'{job.pbar}')
    job.update_progress()

    epoch += 1
    time.sleep(2.0)    
    
job.stop()
pbar.close()

 80%|###################2    | 8/10 [00:16<00:03,  1.92s/it]
{'status': 'done'}


NameError: name 'pbar' is not defined